Crawl dữ liệu từ shopee theo từng category

In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd

PATH = "C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe"


In [2]:
driver = webdriver.Edge(PATH)
# get tạm để lấy cache
driver.get('https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-Nokia-6300-%C4%91%E1%BA%A7y-%C4%91%E1%BB%A7-pin-s%E1%BA%A1c-m%C3%A0u-b%E1%BA%A1c-i.39413762.5162298482')

In [3]:
search_list = pd.read_csv('search.csv',sep='\t')
search_list.to_csv('search_withid.csv',sep='\t',index_label='id')
search_list = pd.read_csv('search_withid.csv',sep='\t')
search_list.head()

,id,name,category,n_sold,price,shop_address,image_url,url
0,0,Điện thoại Xiaomi Redmi Note 9S 4GB/64GB - Hàn...,DIENTHOAI,5600.0,5490000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/b982abe468e4813d2ba0...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
1,1,Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - B...,DIENTHOAI,7800.0,1299000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/654146756efe4b5fe7cf...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
2,2,điện thoại Samsung Galaxy S7 chính hãng / full...,DIENTHOAI,6600.0,1899000.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/622be579c39efa0ea92b...,https://shopee.vn/%C4%91i%E1%BB%87n-tho%E1%BA%...
3,3,Điện thoại Nokia 1280 Huyền thoại chính hãng k...,DIENTHOAI,4100.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...
4,4,Điện thoại Nokia 1280 Nguyên zin Mới tinh chín...,DIENTHOAI,8800.0,189900.0,TP. Hồ Chí Minh,https://cf.shopee.vn/file/c92a82f0a1e8076fcc1e...,https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%...


In [4]:
def reformate_numeric(x):
    if x[-1] == 'k':
        x = float(x.replace(",",".").strip("k")) * 1000
    else:
        x = float(x)
    return x

In [5]:
from cssselector import *
missing_i = []
f = open('items.csv', 'w', encoding="utf-8")
f.write(f'id\tname\tavg_rating\tn_reviews\tn_sold\tprice\tn_loved\tn_rate_5\trate_4\trate_3\trate_2\trate_1\trate_with_cmt\trate_with_imgvid\tshop_name\tshop_n_review\tshop_n_product\tshop_rate_feedback\tshop_time_feedback\tshop_age\tshop_follower\n')
for i,search_item in tqdm(search_list.iterrows(),total=len(search_list),position=0, leave=True):
    try:
        curl = search_item.url
        driver.get(curl)
        sleep(1)
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for j in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(j))
            sleep(0.5)

        # name = driver.find_element_by_css_selector(CSSSelector.NAME).text
        name = search_item['name']
        avg_rating = driver.find_element_by_css_selector(CSSSelector.AVG_RATING).text        
        n_reviews = driver.find_element_by_css_selector(CSSSelector.N_REVIEWS).text        
        n_loved = driver.find_element_by_css_selector(CSSSelector.N_LOVED).text.split(' ')[-1][1:-1]       
        rate_5 = driver.find_element_by_css_selector(CSSSelector.RATE_5).text.split(' ')[-1][1:-1]        
        rate_4 = driver.find_element_by_css_selector(CSSSelector.RATE_4).text.split(' ')[-1][1:-1]
        rate_3 = driver.find_element_by_css_selector(CSSSelector.RATE_3).text.split(' ')[-1][1:-1]
        rate_2 = driver.find_element_by_css_selector(CSSSelector.RATE_2).text.split(' ')[-1][1:-1]
        rate_1 = driver.find_element_by_css_selector(CSSSelector.RATE_1).text.split(' ')[-1][1:-1]
        rate_with_cmt = driver.find_element_by_css_selector(CSSSelector.RATE_WITH_CMT).text.split(' ')[-1][1:-1]
        rate_with_imgvid = driver.find_element_by_css_selector(CSSSelector.RATE_WITH_IMGVID).text.split(' ')[-1][1:-1]
        n_reviews = reformate_numeric(n_reviews)
        n_loved = reformate_numeric(n_loved)
        rate_5 = reformate_numeric(rate_5)
        rate_4 = reformate_numeric(rate_4)
        rate_3 = reformate_numeric(rate_3)
        rate_2 = reformate_numeric(rate_2)
        rate_1 = reformate_numeric(rate_1)
        rate_with_cmt = reformate_numeric(rate_with_cmt)
        rate_with_imgvid = reformate_numeric(rate_with_imgvid)
        n_sold = search_item['n_sold']
        price = search_item['price']
        cid = search_item['id']
        shop_name = driver.find_element_by_css_selector(CSSSelector.SHOP_NAME).text
        
        shop_n_review = driver.find_element_by_css_selector(CSSSelector.SHOP_N_REVIEW).text
        shop_n_product = driver.find_element_by_css_selector(CSSSelector.SHOP_N_PRODUCT).text
        shop_rate_feedback = driver.find_element_by_css_selector(CSSSelector.SHOP_RATE_FEEDBACK).text
        shop_time_feedback = driver.find_element_by_css_selector(CSSSelector.SHOP_TIME_FEEDBACK).text
        shop_age = driver.find_element_by_css_selector(CSSSelector.SHOP_AGE).text
        shop_follower = driver.find_element_by_css_selector(CSSSelector.SHOP_FOLLOWER).text

        shop_n_review = reformate_numeric(shop_n_review)
        shop_n_product = reformate_numeric(shop_n_product)
        shop_follower = reformate_numeric(shop_follower)
        
        f.write(f'{cid}\t{name}\t{avg_rating}\t{n_reviews}\t{n_sold}\t{price}\t{n_loved}\t{rate_5}\t{rate_4}\t{rate_3}\t{rate_2}\t{rate_1}\t{rate_with_cmt}\t{rate_with_imgvid}\t{shop_name}\t{shop_n_review}\t{shop_n_product}\t{shop_rate_feedback}\t{shop_time_feedback}\t{shop_age}\t{shop_follower}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # break
f.close()

100%|██████████| 100/100 [36:56<00:00, 22.17s/it]


In [6]:
missing_i

[20]

In [9]:
from cssselector import *
f = open('items.csv', 'a', encoding="utf-8")
search_list_missing = search_list.iloc[missing_i]
missing_i = []
for i,search_item in tqdm(search_list_missing.iterrows(),total=len(search_list_missing),position=0, leave=True):
    try:
        curl = search_item.url
        driver.get(curl)
        sleep(1)
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for j in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(j))
            sleep(0.5)

        # name = driver.find_element_by_css_selector(CSSSelector.NAME).text
        name = search_item['name']
        avg_rating = driver.find_element_by_css_selector(CSSSelector.AVG_RATING).text        
        n_reviews = driver.find_element_by_css_selector(CSSSelector.N_REVIEWS).text        
        n_loved = driver.find_element_by_css_selector(CSSSelector.N_LOVED).text.split(' ')[-1][1:-1]       
        rate_5 = driver.find_element_by_css_selector(CSSSelector.RATE_5).text.split(' ')[-1][1:-1]        
        rate_4 = driver.find_element_by_css_selector(CSSSelector.RATE_4).text.split(' ')[-1][1:-1]
        rate_3 = driver.find_element_by_css_selector(CSSSelector.RATE_3).text.split(' ')[-1][1:-1]
        rate_2 = driver.find_element_by_css_selector(CSSSelector.RATE_2).text.split(' ')[-1][1:-1]
        rate_1 = driver.find_element_by_css_selector(CSSSelector.RATE_1).text.split(' ')[-1][1:-1]
        rate_with_cmt = driver.find_element_by_css_selector(CSSSelector.RATE_WITH_CMT).text.split(' ')[-1][1:-1]
        rate_with_imgvid = driver.find_element_by_css_selector(CSSSelector.RATE_WITH_IMGVID).text.split(' ')[-1][1:-1]
        n_reviews = reformate_numeric(n_reviews)
        n_loved = reformate_numeric(n_loved)
        rate_5 = reformate_numeric(rate_5)
        rate_4 = reformate_numeric(rate_4)
        rate_3 = reformate_numeric(rate_3)
        rate_2 = reformate_numeric(rate_2)
        rate_1 = reformate_numeric(rate_1)
        rate_with_cmt = reformate_numeric(rate_with_cmt)
        rate_with_imgvid = reformate_numeric(rate_with_imgvid)
        n_sold = search_item['n_sold']
        price = search_item['price']
        cid = search_item['id']
        shop_name = driver.find_element_by_css_selector(CSSSelector.SHOP_NAME).text
        
        shop_n_review = driver.find_element_by_css_selector(CSSSelector.SHOP_N_REVIEW).text
        shop_n_product = driver.find_element_by_css_selector(CSSSelector.SHOP_N_PRODUCT).text
        shop_rate_feedback = driver.find_element_by_css_selector(CSSSelector.SHOP_RATE_FEEDBACK).text
        shop_time_feedback = driver.find_element_by_css_selector(CSSSelector.SHOP_TIME_FEEDBACK).text
        shop_age = driver.find_element_by_css_selector(CSSSelector.SHOP_AGE).text
        shop_follower = driver.find_element_by_css_selector(CSSSelector.SHOP_FOLLOWER).text

        shop_n_review = reformate_numeric(shop_n_review)
        shop_n_product = reformate_numeric(shop_n_product)
        shop_follower = reformate_numeric(shop_follower)
        
        f.write(f'{cid}\t{name}\t{avg_rating}\t{n_reviews}\t{n_sold}\t{price}\t{n_loved}\t{rate_5}\t{rate_4}\t{rate_3}\t{rate_2}\t{rate_1}\t{rate_with_cmt}\t{rate_with_imgvid}\t{shop_name}\t{shop_n_review}\t{shop_n_product}\t{shop_rate_feedback}\t{shop_time_feedback}\t{shop_age}\t{shop_follower}\n')
    except:
        # print('Missing ',i)
        missing_i.append(i)
    #     break
    # break
f.close()

100%|██████████| 1/1 [00:24<00:00, 24.97s/it]


In [13]:
pd.read_csv('items.csv', sep='\t').sort_values(by=['id'], ascending=True, axis=0).to_csv('items_sorted_id.csv',sep='\t',index=False)